# Image Planes

This notebook finds studies whose series all share the same image plane. It produces two sets of output:
1. A list of study IDs that have all 4 planes identical.
1. For each study in the list, a plot of the "orbit" that each patient follows.

See also [David Roberts' notebook](https://www.kaggle.com/davidbroberts/rsna-brain-tumor-image-planes/comments)

I would expect orbits to be parallel if they are in the same plane, but this isn't always true, as shown here.![00098](https://www.kaggleusercontent.com/kf/68577251/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..57ixQ1Ar7KBK3hwjvpxf7A.eXaHTiBHgAjhmlJfzGV_yZFc57CxYh9x8jwrKV8PhVFPMp160yNMvdbmOfvTTFr4dB_B52UQxWcQ_MGWgZk1m6qnL2q1qPxN5c01nbDRl5qvrlZO9adxgevdoOjESrbtYesSc_PO5uMEBjOptAG6prpsG8vnuRKMCDk0oGxSTHc3yeW_pIFAkBZRcy5soJQgNp9AWVPI-pttSRe9HH9sgnUpZcHHWcMAcLl9dajDF7-slro0drBiwaVWNEAWrfepex6kp_CZprCkcErJ5EwXOVOVp8kImBaMVnzpawnRYj5Wa-sbO3-H_GjUdUDh87ctB90_f-ct386ijCKb7hJiVPfyk2af3ipUD1UjrM7hucsGHXOGOHCAJrnk3lLusIqZtDscV0105EZZvnEZhTbnbF6KhV8EBH85ByqBG2BmunPaFUI4iIwY3IYjH7033_QH4en_CidOaF1qwmGUjzHMkxK8QBbGsvGvLtmpPpGUcFW9VxB4d7ZEztMgTs0rmbispIA4FEg2tAP-LOis3EQghR0-hx0tMFgb8QIG2_WV5jwp7q9vRCGc_a9ltdBarHkL6MdU6uW_71Yw5D5wNxBX_NIF4XckcnpdEmJzrwq0o36FBUCv4yqhh9ftITw3QWt0iXx5mDewobDhVzG-4ZtDQdwOg4OcqiY8-PI3VhaXnyM.TZpWwxtK40CvEfGYjqh_VQ/00098.png)

## A few useful imports

In [ ]:
from matplotlib.pyplot import axes, close, cm, figure, savefig, show, title
from pydicom           import dcmread
from os                import sep, listdir, walk
from os.path           import join, normpath
from pandas            import read_csv
from re                import match


## Classes to represent Studies and Datasets

In [ ]:
# Study
#
# This class represents a study for one patient
#
class Study:
    # Series
    #
    # Each Study comprises several (in practic 4) Series
    class Series:
        def __init__(self,name):
            self.name             = name
            self.missing_images   = set()
            self.dirpath          = None
            self.image_plane      = None
            self.description      = None
            self.patient_position = None

        # add_images
        #
        # Add a collection of images to Series

        def add_images(self,dirpath,filenames):
            def extract_digits(s):
                m = match(r'\D*(\d+)\D+',s)
                if m:
                    return int(m.group(1))

            self.dirpath          = dirpath
            seqs                  = sorted([extract_digits(name) for name in filenames])
            self.N                = seqs[-1]
            self.missing_images   = set([i for i in range(1,self.N) if i not in seqs])
            dcim                  = dcmread(join(dirpath,filenames[0]))
            self.image_plane      = self.get_image_plane(dcim.ImageOrientationPatient)
            self.description      = dcim.SeriesDescription
            self.patient_position = dcim.PatientPosition

        # dcmread
        #
        # A generator to iterate through all the images.
        #
        # parameters:
        #     stop_before_pixels    Used if we just want to analyze metadata

        def dcmread(self, stop_before_pixels = False):
            for i in range(1,len(self)+1):
                if i not in self.missing_images:
                    yield  dcmread(join(self.dirpath,f'Image-{i}.dcm'), stop_before_pixels = stop_before_pixels)

        # get_image_plane
        #
        # Snarfed from https://www.kaggle.com/davidbroberts/determining-mr-image-planes
        def get_image_plane(self,loc):
            row_x = round(loc[0])
            row_y = round(loc[1])
            row_z = round(loc[2])
            col_x = round(loc[3])
            col_y = round(loc[4])
            col_z = round(loc[5])

            if row_x == 1 and row_y == 0 and col_x == 0 and col_y == 0:  return "Coronal"

            if row_x == 0 and row_y == 1 and col_x == 0 and col_y == 0:  return "Sagittal"

            if row_x == 1 and row_y == 0 and col_x == 0 and col_y == 1:  return "Axial"

            return "Unknown"

        def __len__(self):
            return self.N

        # image_files
        #
        # Generator for iterating through image files

        def image_files(self):
            for i in range(1,self.N+1):
                if i not in self.missing_images:
                    yield join(self.dirpath,f'Image-{i}.dcm')

    def __init__(self,name,path):
        self.series        = None
        self.name          = name
        for dirpath, dirnames, filenames in walk(path):
            if self.series == None:
                self.series = {series_name: Study.Series(series_name) for series_name in dirnames}
            else:
                path_components = normpath(dirpath).split(sep)
                series = self.series[path_components[-1]]
                series.add_images(dirpath,filenames)

    def get_series(self):
        for name in ['FLAIR', 'T1w', 'T1wCE', 'T2w']:
            yield self.series[name]

    def get_image_planes(self):
        def get_image_plane(series):
            path_name   = next(series.image_files())
            dcim        = dcmread(path_name,stop_before_pixels=True)
            return  series.get_image_plane(dcim.ImageOrientationPatient)
        return [get_image_plane(series)  for series in self.series.values()]

    def __str__(self):
        return self.name

# MRI_Dataset
#
# An MRI Dataset comprises sevral stidies, either test or training

class MRI_Dataset:
    def __init__(self,path,folder):
        self.studies = {name:Study(name,join(path,folder,name)) for name in listdir(join(path,folder))}

    def get_studies(self):
        for study in self.studies.values():
            yield study

# Labelled_MRI_Dataset
#
# A Labelled_MRI_Dataset is a MRI_Dataset accompanied by labels for training

class Labelled_MRI_Dataset(MRI_Dataset):

    def __init__(self,path,folder,labels='train_labels.csv'):
        super().__init__(path,folder)
        self.labels = read_csv(join(path,labels),dtype={'BraTS21ID':str})

In [ ]:
# get_values_from_meta
#
# Convert meta data (list of tuples of ASCII data) to a list of data
# sequences, each being the time series for one datum

def get_values_from_meta(orbit):
    return [[float(a) for a in p] for p in list(zip(*orbit))]

# plot_orbit
#
# Show how the patient moves through the MRI instrument during one Study

def plot_orbit(study):
    fig       = figure(figsize=(20,20))
    ax        = axes(projection='3d')

    for series in study.get_series():
        orbit = []
        sizes = []
        for dcim in series.dcmread():
            orbit.append(dcim.ImagePositionPatient)
            sizes.append(10 if dcim.pixel_array.sum()> 0 else 1)
        ax.scatter(*get_values_from_meta(orbit),
                   label = f'{dcim.SeriesDescription}: {dcim.PatientPosition} {series.get_image_plane(dcim.ImageOrientationPatient)}',
                   s     = sizes)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    title(dcim.PatientID)
    ax.legend()
    savefig(dcim.PatientID)
    return fig


In [ ]:
path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'
training = Labelled_MRI_Dataset(path,'train')
with open('unique.csv','w') as out:
    for study in training.get_studies():
        image_planes = study.get_image_planes()
        if len(set(image_planes))==1:
            print (study, image_planes[0])
            out.write(f'{study}, {image_planes[0]}\n')
            fig = plot_orbit(study)
            close(fig)